### Luigi Module

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

# Dealing with multiple outputs

dataset = pars.get('dataset')

In [88]:
import pandas as pd

In [106]:
tagged_dataset_file = '../../data/stack-overflow/Dataset - Pandas.csv'
tagged_dataset_df = pd.read_csv(tagged_dataset_file, encoding='ISO-8859-1', error_bad_lines=False)

result_df = pd.read_pickle('../../data/results/result_df_m1')


In [118]:
def buildTaggedDatasetDSForEvaluation(tagged_dataset_df):
    taggedDSEvalDict = dict()
    try:
        for index, row in tagged_dataset_df.iterrows():
            Id = row['AnswerId']
            if Id != 0:
                
                if Id in taggedDSEvalDict:
                    solutionList = taggedDSEvalDict[Id]
                else:
                    solutionList = list()                    
                lineList = list()
                lines = row['Solution'].split('\n')
                for line in lines:
                    if line.strip() != '':
                        pair = [line, False]
                        lineList.append(pair)
                solutionList.append(lineList)
                taggedDSEvalDict[Id]= solutionList
        
    except Exception as e:
        print(Id, e)
    return taggedDSEvalDict


def buildResultSetForEvaluation(result_df):
    resultSetDict = dict()
    try:
        for index, row in result_df.iterrows():
            ansId = row['Id']
            solutionLinesTuple = list()                    
            predictedLines = row['predictedSolution']
            for line in predictedLines:
                if line.strip() != '':
                    pair = [line, False]
                    solutionLinesTuple.append(pair)
            resultSetDict[ansId]= solutionLinesTuple
        
    except Exception as e:
        print(ansId, e)
    return resultSetDict

def markTheLines(taggedDSEvalDict, resultSetDict):
    for Id, pairs in resultSetDict.items():
        solutionList = taggedDSEvalDict[Id]
        for respair in pairs:
            for solution in solutionList:
                for tagpair in solution:
                    if tagpair[0].strip() == respair[0].strip():
                        tagpair[1] = True
                        respair[1] = True
    return taggedDSEvalDict, resultSetDict

In [128]:
def evaluate(result_df):
    TP = 0
    FP = 0
    taggedDSEvalDict =  buildTaggedDatasetDSForEvaluation(tagged_dataset_df)
    resultSetDict = buildResultSetForEvaluation(result_df)
    taggedDSEvalDict, resultSetDict = markTheLines(taggedDSEvalDict, resultSetDict)
    for Id, pairs in resultSetDict.items():
        MA = True
        for pair in pairs:
            MA = MA and pair[1]
        if MA:
            MP = False
            solutions = taggedDSEvalDict[Id]
            for solution in solutions:
                MPS = True
                for tagpair in solution:
                    MPS = MPS and tagpair[1]
                MP = MP or MPS
            if MP:
                TP = TP + 1
            else:
                FP = FP + 1
                    
        else:
            FP = FP + 1
    return float(TP)/(TP+FP)

In [130]:
precision =  evaluate(result_df)
print(precision*100, '%')

17841294 'float' object has no attribute 'split'
50.0 %
